# GraphRAG Patterns

In this notebook, we'll explore advanced GraphRAG patterns that combine vector search with graph traversal. We'll cover:

1. Graph-enhanced retrieval
2. Text2Cypher for natural language queries
3. Making AI decisions explainable

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import openai
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embedder import OpenAIEmbedder
from neo4j_graphrag.retriever import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag.text2cypher import Text2Cypher

# Setup
load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

openai.api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAILLM()
embedder = OpenAIEmbedder()

## 1. Graph-Enhanced Retrieval

Let's see how combining vector search with graph traversal improves results. Our graph contains:
- Products (name, category, price)
- Customers who placed orders
- Orders linking customers to products
- Documents (manuals and support cases)

In [ ]:
# Create vector-cypher retriever for our tech support use case
graph_retriever = VectorCypherRetriever(
    driver=driver,
    embedder=embedder,
    node_label="Product",
    embedding_property="embedding",
    cypher_template="""
    MATCH (p:Product)
    WHERE p.embedding IS NOT NULL
    WITH p, gds.similarity.cosine(p.embedding, $query_embedding) AS score
    WHERE score > 0.7
    
    // Enhance with graph context from our data model
    OPTIONAL MATCH (p)<-[:ORDERED]-(o:Order)<-[:PLACED]-(c:Customer)
    OPTIONAL MATCH (p)<-[:ABOUT]-(d:Document)
    OPTIONAL MATCH (c)-[:PLACED]->(o2:Order)-[:ORDERED]->(p2:Product)
    WHERE p2 <> p
    
    RETURN p.name as product_name,
           p.category as category,
           p.price as price,
           score,
           collect(DISTINCT d.content) as documentation,
           collect(DISTINCT p2.name) as frequently_bought_with,
           count(DISTINCT c) as customer_count
    ORDER BY score DESC
    LIMIT 3
    """
)

# Example queries using our product data
queries = [
    "Find accessories commonly purchased with the Laptop Pro",
    "Show me storage solutions with good customer satisfaction",
    "What products are recommended for productivity setup?"
]

for query in queries:
    print(f"\nQuery: {query}")
    results = graph_retriever.retrieve(query)
    
    print("Graph-enhanced results (with context):")
    for r in results:
        print(f"\nProduct: {r.product_name}")
        print(f"Category: {r.category}")
        print(f"Price: ${r.price}")
        print(f"Relevance Score: {r.score:.2f}")
        print(f"Documentation: {r.documentation[:200]}...")  # Truncate long text
        print(f"Frequently Bought With: {r.frequently_bought_with}")
        print(f"Number of Customers: {r.customer_count}")

## 2. Text2Cypher for Natural Language Queries

Convert natural language to Cypher queries that understand our data model:

In [ ]:
# Initialize Text2Cypher with our schema
text2cypher = Text2Cypher(
    driver=driver,
    llm=llm,
    schema_hint="""
    The graph contains:
    - Products (name, category, price)
    - Customers who placed orders
    - Orders linking customers to products
    - Documents (manuals and support cases)
    
    Common relationships:
    - (Customer)-[:PLACED]->(Order)-[:ORDERED]->(Product)
    - (Document)-[:ABOUT]->(Product)
    """
)

# Natural language query examples based on our data
nl_queries = [
    "What products do customers usually buy after purchasing the Laptop Pro?",
    "Find products mentioned in support cases with positive feedback",
    "Show me the most popular accessories based on order history"
]

for query in nl_queries:
    print(f"\nNatural Language Query: {query}")
    cypher = text2cypher.translate(query)
    print(f"Generated Cypher:\n{cypher}")
    
    with driver.session() as session:
        results = session.run(cypher)
        for record in results:
            print(f"Result: {record}")

## 3. Making AI Decisions Explainable

Let's make our recommendations transparent by explaining how we use graph data:

In [ ]:
def explain_recommendation(query, results):
    explanation_prompt = f"""
    Query: {query}
    
    Results:
    {results}
    
    Explain why these products were recommended, considering:
    1. Product features and categories
    2. Customer purchase patterns
    3. Support case feedback
    4. Technical compatibility
    """
    
    explanation = llm.generate(explanation_prompt)
    return explanation

# Example usage
query = "What should I buy to enhance my Laptop Pro setup?"
results = graph_retriever.retrieve(query)

explanation = explain_recommendation(query, results)
print("\nExplanation of recommendations:")
print(explanation)